In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/depi/Final Project/Week 3&4 - Advanced models/NLP/FraudNLP_dataset (1).csv')
df.head()

,actions_str,actions,execution_time,Amount,type,is_fraud
0,/PROFILE/GETCUSTOMERRESPONSE /TAXFREE/GETTAXGO...,"[22, 27, 24, 1, 1268, 1269, 1267, 22, 29, 1, 2...",203,13,CASH_OUT,0
1,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 1, 1268, 1269, 1267, 2, 23, 6, 25...",359,310,PAYMENT,0
2,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/USERPROF...,"[22, 1, 29, 22, 26, 2, 23, 25, 6, 28, 14, 7, 6...",250,350,CASH_OUT,0
3,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 72, 1269, 1268, 1267, 4, 70, 46, ...",203,350,CASH_OUT,0
4,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 23, 2, 1269, 1269, 6, 25, 7, 28, ...",593,2000,CASH_IN,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df[['actions_str','actions','execution_time', 'Amount', 'type', 'is_fraud']]
df

,actions_str,actions,execution_time,Amount,type,is_fraud
0,/PROFILE/GETCUSTOMERRESPONSE /TAXFREE/GETTAXGO...,"[22, 27, 24, 1, 1268, 1269, 1267, 22, 29, 1, 2...",203,13,CASH_OUT,0
1,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 1, 1268, 1269, 1267, 2, 23, 6, 25...",359,310,PAYMENT,0
2,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/USERPROF...,"[22, 1, 29, 22, 26, 2, 23, 25, 6, 28, 14, 7, 6...",250,350,CASH_OUT,0
3,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 72, 1269, 1268, 1267, 4, 70, 46, ...",203,350,CASH_OUT,0
4,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 23, 2, 1269, 1269, 6, 25, 7, 28, ...",593,2000,CASH_IN,1
...,...,...,...,...,...,...
105297,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/GETCUSTO...,"[22, 22, 27, 24, 17, 15, 1, 16, 53, 2, 23, 28,...",265,10,PAYMENT,0
105298,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/GETCUSTO...,"[22, 22, 24, 27, 1, 16, 17, 53, 15, 29, 22, 26...",328,3000,CASH_IN,0
105299,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/GETCUSTO...,"[22, 22, 24, 27, 1, 15, 17, 16, 53, 23, 2, 28,...",265,8,CASH_OUT,0
105300,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/GETCUSTO...,"[22, 22, 24, 27, 1, 7, 1, 14, 24, 27, 33, 39, ...",265,70,CASH_OUT,0


In [ ]:
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
df.head()

,actions_str,actions,execution_time,Amount,type,is_fraud
0,/PROFILE/GETCUSTOMERRESPONSE /TAXFREE/GETTAXGO...,"[22, 27, 24, 1, 1268, 1269, 1267, 22, 29, 1, 2...",203,13,1,0
1,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 1, 1268, 1269, 1267, 2, 23, 6, 25...",359,310,2,0
2,/PROFILE/GETCUSTOMERRESPONSE /PROFILE/USERPROF...,"[22, 1, 29, 22, 26, 2, 23, 25, 6, 28, 14, 7, 6...",250,350,1,0
3,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 72, 1269, 1268, 1267, 4, 70, 46, ...",203,350,1,0
4,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...,"[22, 24, 27, 23, 2, 1269, 1269, 6, 25, 7, 28, ...",593,2000,0,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
actions_matrix = vectorizer.fit_transform(df['actions'].apply(lambda x: [str(i) for i in x])).toarray()

# Combine the `actions_matrix` with other numeric features (`execution_time` and `Amount`)
X = np.hstack([actions_matrix, df[['execution_time', 'Amount','type']].values])
y = df['is_fraud']

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X

array([[  23,   23,    0, ...,  203,   13,    1],
       [2982, 2982,    4, ...,  359,  310,    2],
       [ 222,  222,    6, ...,  250,  350,    1],
       ...,
       [ 106,  106,    0, ...,  265,    8,    1],
       [  66,   66,    0, ...,  265,   70,    1],
       [ 204,  204,    6, ...,  374,   10,    3]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.value_counts()

,count
is_fraud,
0,84157
1,84


In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
y_train_smote.value_counts()

,count
is_fraud,
0,84157
1,84157


In [ ]:
print(X_train_smote.shape)

(168314, 17)


In [ ]:
X_train_smote

array([[  10,   10,    2, ...,  782,  121,    1],
       [  32,   32,    0, ...,  656,  472,    0],
       [ 250,  250,    0, ...,  640,  300,    2],
       ...,
       [ 223,  223,   13, ...,  624, 3085,    0],
       [ 145,  145,    2, ...,  534, 9000,    1],
       [ 541,  541,    5, ...,  642, 9974,    0]])

In [ ]:
sm = SMOTE(random_state=42)
X_test_smote, y_test_smote = smote.fit_resample(X_test, y_test)
y_test_smote.value_counts()

,count
is_fraud,
0,21044
1,21044


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

num_features = X_train_smote.shape[1]
X_train_smote_reshaped = X_train_smote.reshape(X_train_smote.shape[0], 1, num_features)
X_test_reshaped = X_test_smote.reshape(X_test_smote.shape[0], 1, num_features)

model = Sequential()
model.add(LSTM(units=64, input_shape=(1, num_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_smote_reshaped, y_train_smote, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4734/4734 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9398 - loss: 0.1850 - val_accuracy: 0.9769 - val_loss: 0.0995
Epoch 2/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9566 - loss: 0.1312 - val_accuracy: 0.9955 - val_loss: 0.0434
Epoch 3/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9671 - loss: 0.1055 - val_accuracy: 0.9521 - val_loss: 0.1314
Epoch 4/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9679 - loss: 0.1028 - val_accuracy: 0.9995 - val_loss: 0.0496
Epoch 5/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9699 - loss: 0.0982 - val_accuracy: 0.9963 - val_loss: 0.0420
Epoch 6/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9710 - loss: 0.0961 - val_accuracy: 1.0000 - val_loss: 0.0436
Epoch 7/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9697 - loss: 0.0999 - val_accuracy: 0.9969 - val_loss: 0.0463
Epoch 8/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9742 - loss: 0.0894 - val

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = (model.predict(X_test_reshaped) > 0.5).astype("int32")
accuracy = accuracy_score(y_test_smote, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_smote, y_pred))

1316/1316 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Accuracy: 0.9847462459608439
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     21044
           1       0.97      1.00      0.98     21044

    accuracy                           0.98     42088
   macro avg       0.99      0.98      0.98     42088
weighted avg       0.99      0.98      0.98     42088



In [ ]:
y_pred_ = (model.predict(X_train_smote_reshaped) > 0.5).astype("int32")  # Predictions on training data
accuracy_ = accuracy_score(y_train_smote, y_pred_)                      # Compare with training labels
print("Accuracy:", accuracy_)
print(classification_report(y_train_smote, y_pred_))

5260/5260 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step
Accuracy: 0.9829069477286501
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     84157
           1       0.97      1.00      0.98     84157

    accuracy                           0.98    168314
   macro avg       0.98      0.98      0.98    168314
weighted avg       0.98      0.98      0.98    168314



In [ ]:
df.iloc[0]

,0
actions_str,/PROFILE/GETCUSTOMERRESPONSE /TAXFREE/GETTAXGO...
actions,"[22, 27, 24, 1, 1268, 1269, 1267, 22, 29, 1, 2..."
execution_time,203
Amount,13
type,1
is_fraud,0


In [ ]:
new_instance = df.iloc[0]
new_instance.drop(['is_fraud', 'actions_str'], inplace=True)
new_instance

<ipython-input-93-eb81cb4a999c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_instance.drop(['is_fraud', 'actions_str'], inplace=True)


,0
actions,"[22, 27, 24, 1, 1268, 1269, 1267, 22, 29, 1, 2..."
execution_time,203
Amount,13
type,1


In [ ]:
new_actions_matrix = vectorizer.transform([new_instance['actions']]).toarray()
new_instance_features = np.hstack([new_actions_matrix, [[new_instance['execution_time'], new_instance['Amount'], new_instance['type']]]])
new_instance_reshaped = new_instance_features.reshape(1, 1, new_instance_features.shape[1])
fraud_probability = model.predict(new_instance_reshaped)[0][0]

threshold = 0.5
if fraud_probability > threshold:
    output = 1  # Fraud
else:
    output = 0  # Not fraud

print("Output:", output)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Output: 0


In [ ]:
df.iloc[4]

,4
actions_str,/PROFILE/GETCUSTOMERRESPONSE /CAMPAIGN/GETBALA...
actions,"[22, 24, 27, 23, 2, 1269, 1269, 6, 25, 7, 28, ..."
execution_time,593
Amount,2000
type,0
is_fraud,1


In [ ]:
new_inst = df.iloc[4]
new_inst.drop(['is_fraud', 'actions_str'], inplace=True)
new_inst

<ipython-input-96-8e79a8f3bc42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inst.drop(['is_fraud', 'actions_str'], inplace=True)


,4
actions,"[22, 24, 27, 23, 2, 1269, 1269, 6, 25, 7, 28, ..."
execution_time,593
Amount,2000
type,0


In [ ]:
new_actions_matrix = vectorizer.transform([new_inst['actions']]).toarray()
new_instance_features = np.hstack([new_actions_matrix, [[new_inst['execution_time'], new_inst['Amount'], new_inst['type']]]])
new_instance_reshaped = new_instance_features.reshape(1, 1, new_instance_features.shape[1])
fraud_probability = model.predict(new_instance_reshaped)[0][0]

threshold = 0.5
if fraud_probability > threshold:
    output = 1  # Fraud
else:
    output = 0  # Not fraud

print("Output:", output)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Output: 1


In [ ]:
#GRU

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

num_features = X_train_smote.shape[1]
X_train_smote_reshaped = X_train_smote.reshape(X_train_smote.shape[0], 1, num_features)
X_test_reshaped = X_test_smote.reshape(X_test_smote.shape[0], 1, num_features)

model = Sequential()
model.add(GRU(units=64, input_shape=(1, num_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_smote_reshaped, y_train_smote, epochs=10, batch_size=32, validation_split=0.1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9365 - loss: 0.1736 - val_accuracy: 0.9619 - val_loss: 0.1553
Epoch 2/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9583 - loss: 0.1238 - val_accuracy: 0.9949 - val_loss: 0.0500
Epoch 3/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9692 - loss: 0.1008 - val_accuracy: 0.9997 - val_loss: 0.0558
Epoch 4/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9681 - loss: 0.1036 - val_accuracy: 0.9874 - val_loss: 0.0652
Epoch 5/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9731 - loss: 0.0905 - val_accuracy: 0.9996 - val_loss: 0.0423
Epoch 6/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9717 - loss: 0.0955 - val_accuracy: 0.9996 - val_loss: 0.0363
Epoch 7/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9769 - loss: 0.0830 - val_accuracy: 0.9994 - val_loss: 0.0245
Epoch 8/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9771 - loss: 0

In [ ]:
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Reshape y_pred to match y_test_smote
y_pred = y_pred.reshape(y_test_smote.shape) # Reshape to match y_test_smote

# Check and potentially fix data types
y_test_smote = y_test_smote.astype(int)  # Ensure y_test_smote is of type int
y_pred = y_pred.astype(int)  # Ensure y_pred is of type int

accuracy = accuracy_score(y_test_smote, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_smote, y_pred))

1316/1316 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Accuracy: 0.9831305835392511
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     21044
           1       0.97      1.00      0.98     21044

    accuracy                           0.98     42088
   macro avg       0.98      0.98      0.98     42088
weighted avg       0.98      0.98      0.98     42088



In [ ]:
y_pred_probs = model.predict(X_train_smote_reshaped)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Reshape y_pred to match y_test_smote
y_pred = y_pred.reshape(y_train_smote.shape) # Reshape to match y_test_smote

# Check and potentially fix data types
y_train_smote = y_train_smote.astype(int)  # Ensure y_test_smote is of type int
y_pred = y_pred.astype(int)  # Ensure y_pred is of type int

accuracy = accuracy_score(y_train_smote, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_train_smote, y_pred))

5260/5260 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
Accuracy: 0.9850755136233469
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     84157
           1       0.97      1.00      0.99     84157

    accuracy                           0.99    168314
   macro avg       0.99      0.99      0.99    168314
weighted avg       0.99      0.99      0.99    168314



In [ ]:
#Fully Connected Neural Network (FCNN)

In [ ]:

model = Sequential()
model.add(Dense(units=128, input_shape=(1, num_features), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_smote_reshaped, y_train_smote, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4734/4734 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9221 - loss: 1.2201 - val_accuracy: 0.9406 - val_loss: 0.9552
Epoch 2/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9296 - loss: 1.1232 - val_accuracy: 0.9354 - val_loss: 1.0401
Epoch 3/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9290 - loss: 1.1371 - val_accuracy: 0.9130 - val_loss: 1.3997
Epoch 4/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9303 - loss: 1.1164 - val_accuracy: 0.9380 - val_loss: 0.9997
Epoch 5/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9341 - loss: 1.0553 - val_accuracy: 0.9162 - val_loss: 1.3509
Epoch 6/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9347 - loss: 1.0445 - val_accuracy: 0.9184 - val_loss: 1.3117
Epoch 7/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9290 - loss: 1.1382 - val_accuracy: 0.9354 - val_loss: 1.0419
Epoch 8/10
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9344 - loss: 1.0502 - val

In [ ]:
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Reshape y_pred to match y_test_smote
y_pred = y_pred.reshape(y_test_smote.shape) # Reshape to match y_test_smote

# Check and potentially fix data types
y_test_smote = y_test_smote.astype(int)  # Ensure y_test_smote is of type int
y_pred = y_pred.astype(int)  # Ensure y_pred is of type int

accuracy = accuracy_score(y_test_smote, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_smote, y_pred))

1316/1316 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Accuracy: 0.952528036494963
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     21044
           1       0.94      0.97      0.95     21044

    accuracy                           0.95     42088
   macro avg       0.95      0.95      0.95     42088
weighted avg       0.95      0.95      0.95     42088



In [ ]:
y_pred_probs = model.predict(X_train_smote_reshaped)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Reshape y_pred to match y_test_smote
y_pred = y_pred.reshape(y_train_smote.shape) # Reshape to match y_test_smote

# Check and potentially fix data types
y_train_smote = y_train_smote.astype(int)  # Ensure y_test_smote is of type int
y_pred = y_pred.astype(int)  # Ensure y_pred is of type int

accuracy = accuracy_score(y_train_smote, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_train_smote, y_pred))

5260/5260 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
Accuracy: 0.9358698622812125
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     84157
           1       0.94      0.93      0.94     84157

    accuracy                           0.94    168314
   macro avg       0.94      0.94      0.94    168314
weighted avg       0.94      0.94      0.94    168314

